In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [2]:
def loadImage(file_path, read_param):
    """
    @param file_path: image file path
    @param read_param: mode of loading the image (gray, colored etc.)
    
    @return ret: loaded image
    """
    
    param_dict = {'gray': cv2.IMREAD_GRAYSCALE, 'color': cv2.IMREAD_COLOR}#, 'with_alpha': cv2.IMREAD_UNCHANGED}
    ret = cv2.imread(file_path, param_dict[read_param])

    if ret is None:
        print('check file path')
        return

    if read_param == 'color':
        ret = cv2.cvtColor(ret, cv2.COLOR_BGR2RGB) 

    print('image', file_path, 'loaded')
    return ret

In [4]:
def blurLegend(directions, magnitude):
    """
    @param directions: matrix containing blur directions
    @param magnitude: matrix containg blur magnitude
    
    function for visualising blur matrix (blur in the image)
    """
    norm = mpl.colors.Normalize(0, np.pi)
    plt.imshow(norm(directions))
    plt.show()

In [5]:
def rotateMatrix(matrix, angle):
    """
    @param matrix: input matrix
    @param angle: rotation angle
    
    @return rotmat: input matrix rotated by angle theta(param)
    """
    from skimage.transform import rotate
    return rotate(matrix, angle)

In [6]:
def MSELoss(img1, img2):
    """
    @param img1: image 1
    @param img2: image 2
    
    @return loss: mse loss between image1 and image 2
    """
    
    return np.sum(np.power(img1 - img2, 2))

In [1]:
def blurKernel(r, theta):
    """
    @param r: blur kernel diameter
    @param theta: blur direction
    
    @return kernel: blur kernel b(r, theta)
    """
    
    kernel = np.zeros((r,r))
    kernel[r//2,:] = 1
    kernel = rotateMatrix(kernel, theta)
    return kernel/np.sum(kernel)

In [11]:
def linearBlur(img, r, theta):
    """
    @param img: input image
    @param r: blur kernel radius
    @param theta: blur direction
    
    @return image convolved with blur kernel b(r, theta)
    """
    kernel = blurKernel(r, theta)
    return cv2.filter2D(np.uint8(img), -1, kernel)

In [12]:
def nonUniformBlurKernels(img):
    """
    @param input: img (nxm)
    
    @return K: list of filters for each pixel
    """
    
    K = []
    for i in tqdm(range(img.shape[0])):
        for j in range(img.shape[1]):
            r = 1+min(2*(img.shape[0]-1-i)+1, 2*(img.shape[1]-1-j)+1, 20, 2*i+1, 2*j+1)
            if r%2==0:
                r-=1
                
            theta = 45 #np.random.randint(0,180)
            kernel = blurKernel(r,theta)

            K.append(kernel)
            
    return np.array(K)
            
def applyNonUniformBlur(img, K):
    """
    @param img: (nxm)
    @param K: list of kernels (nxm kernels)
    
    @return blurred_img
    """
    
    blurred_img = np.zeros(img.shape)
    n,m = img.shape
    
    for i in range(n):
        for j in range(m):
            kernel = K[i*m+j]
            k = kernel.shape[0]
            img_patch = img[i-k//2:i+k//2+1, j-k//2:j+k//2+1]
            blurred_img[i,j] = np.sum(img_patch*kernel)
            
    return blurred_img
    
def nonUniformBlur(img):
    """
    @param img: (nxm)
    
    @return blurred_img: (nxm)
    @return K: list of blur kernels (n*m)
    """
    
    K = nonUniformBlurKernels(img)
    blurred_img = applyNonUniformBlur(img, K)
    return blurred_img, K

In [8]:
def im2col(A, sz):
    """
    @param A: input image
    @param sz: size of blocks
    
    @return out: columised image (same working as im2col of matlab)
    """
    
    m, n = A.shape
    s1, s2 = A.strides
    rows = m-sz[0]+1
    cols = n-sz[1]+1
    shp = sz[0],sz[1],rows,cols
    strd = s1,s2,s1,s2

    out = np.lib.stride_tricks.as_strided(A, shape=shp, strides=strd).reshape(sz[0]*sz[1],-1)[:,::1]
    return out